# *📊 Employee Attrition - Data Preparation*

---

## *Author*  
**Kfir Tayar**  

## *Notebook Overview*
- Check if there are duplicate records.
- Check if there are missing values. 
- Remove punctuation. 
- Reduce large category features. 
- Drop unnecessary or illogical features.   
- Save the prep file into pickle format.


In [1]:
# Import Libraries & Modules
import sys
import os
import pandas as pd
import numpy as np

# Add the path to the utils directory
sys.path.append(os.path.abspath('../utils'))

from data_prep_utils import save_file_as_pickle, drop_selected_cols, display_category_summary, turn_object_into_category

### Load Data Sets

In [4]:
df1 = pd.read_csv(r"../data/train.csv")
df2 = pd.read_csv(r"../data/test.csv")

In [6]:
# Combine train & test data sets
df = pd.concat([df1, df2], ignore_index=True)

In [8]:
df.head()

,Employee ID,Age,Gender,Years at Company,Job Role,Monthly Income,Work-Life Balance,Job Satisfaction,Performance Rating,Number of Promotions,...,Number of Dependents,Job Level,Company Size,Company Tenure,Remote Work,Leadership Opportunities,Innovation Opportunities,Company Reputation,Employee Recognition,Attrition
0,8410,31,Male,19,Education,5390,Excellent,Medium,Average,2,...,0,Mid,Medium,89,No,No,No,Excellent,Medium,Stayed
1,64756,59,Female,4,Media,5534,Poor,High,Low,3,...,3,Mid,Medium,21,No,No,No,Fair,Low,Stayed
2,30257,24,Female,10,Healthcare,8159,Good,High,Low,0,...,3,Mid,Medium,74,No,No,No,Poor,Low,Stayed
3,65791,36,Female,7,Education,3989,Good,High,High,1,...,2,Mid,Small,50,Yes,No,No,Good,Medium,Stayed
4,65026,56,Male,41,Education,4821,Fair,Very High,Average,0,...,0,Senior,Medium,68,No,No,No,Fair,Medium,Stayed


In [10]:
# Chack if there is duplicated rows
(len(df[df.duplicated()]))

0

In [12]:
# Chack if there is NaN values
sum(df.isna().sum())

0

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74498 entries, 0 to 74497
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Employee ID               74498 non-null  int64 
 1   Age                       74498 non-null  int64 
 2   Gender                    74498 non-null  object
 3   Years at Company          74498 non-null  int64 
 4   Job Role                  74498 non-null  object
 5   Monthly Income            74498 non-null  int64 
 6   Work-Life Balance         74498 non-null  object
 7   Job Satisfaction          74498 non-null  object
 8   Performance Rating        74498 non-null  object
 9   Number of Promotions      74498 non-null  int64 
 10  Overtime                  74498 non-null  object
 11  Distance from Home        74498 non-null  int64 
 12  Education Level           74498 non-null  object
 13  Marital Status            74498 non-null  object
 14  Number of Dependents  

In [16]:
# Make all the Object type features into Category type
df = turn_object_into_category(df)

In [18]:
# Convert specific numeric features to categorical features
df['Number of Dependents'] = df['Number of Dependents'].astype('category')
df['Number of Promotions'] = df['Number of Promotions'].astype('category')

In [20]:
# Chack if there is a need to reduce categories
display_category_summary(df)

,Unique Values,Categories
Feature,,
Gender,2,"[Male, Female]"
Job Role,5,"[Education, Media, Healthcare, Technology, Fin..."
Work-Life Balance,4,"[Excellent, Poor, Good, Fair]"
Job Satisfaction,4,"[Medium, High, Very High, Low]"
Performance Rating,4,"[Average, Low, High, Below Average]"
Number of Promotions,5,"[2, 3, 0, 1, 4]"
Overtime,2,"[No, Yes]"
Education Level,5,"[Associate Degree, Master’s Degree, Bachelor’s..."
Marital Status,3,"[Married, Divorced, Single]"


### Remove punctuation

In [23]:
df['Education Level'] = df['Education Level'].str.replace("’", "").astype("category")

In [25]:
df['Education Level'].value_counts()

Education Level
Bachelors Degree    22331
Associate Degree    18649
Masters Degree      15021
High School         14680
PhD                  3817
Name: count, dtype: int64

### Reduce Age Features

In [27]:
# Calculate the age when the employee started at the company
df['Start Age'] = df['Age'] - df['Years at Company']

df['Age Group'] = pd.cut(df['Age'], bins=[18, 23, 30, 40, 50, 60], labels=['18-23', '23-30', '30-40', '40-50', '>50'], right=False)
print(df['Age Group'].value_counts(), end='\n\n')

df['Start Age Group'] = pd.cut(df['Start Age'], bins=[8, 18, 23, 30, 40, 60], labels=['8-18', '18-23', '23-30', '30-40', '>40'], right=False)
print(df['Start Age Group'].value_counts())

Age Group
30-40    17980
40-50    17873
>50      17633
23-30    12180
18-23     8832
Name: count, dtype: int64

Start Age Group
8-18     29812
23-30    12930
18-23    12493
30-40    11731
>40       7532
Name: count, dtype: int64


### Drop Unnecessary Features

In [29]:
# Strange values. An employee can't work 128 years in the industry
df['Company Tenure'].value_counts()

Company Tenure
65     994
43     993
55     986
50     974
52     974
      ... 
123     13
125     12
126      7
127      3
128      1
Name: count, Length: 127, dtype: int64

In [31]:
# More than 50% of the employees have several years in the industry, which is greater than their age (doesn't make sense)
len(df[df['Company Tenure'] > df['Age']])

52902

In [33]:
# Drop Company Tenure
# Drop Employee ID - Unique number
# Drop Age - Prevent multicollinearity
# Drop Strat Age - Prevent multicollinearity
cols_to_drop = ['Company Tenure', 'Employee ID', 'Age', 'Start Age']
df = drop_selected_cols(df, cols_to_drop)

### Prepared data set

In [35]:
df.select_dtypes(include=['number']).head()

,Years at Company,Monthly Income,Distance from Home
0,19,5390,22
1,4,5534,21
2,10,8159,11
3,7,3989,27
4,41,4821,71


In [37]:
df.select_dtypes(['category']).head()

,Gender,Job Role,Work-Life Balance,Job Satisfaction,Performance Rating,Number of Promotions,Overtime,Education Level,Marital Status,Number of Dependents,Job Level,Company Size,Remote Work,Leadership Opportunities,Innovation Opportunities,Company Reputation,Employee Recognition,Attrition,Age Group,Start Age Group
0,Male,Education,Excellent,Medium,Average,2,No,Associate Degree,Married,0,Mid,Medium,No,No,No,Excellent,Medium,Stayed,30-40,8-18
1,Female,Media,Poor,High,Low,3,No,Masters Degree,Divorced,3,Mid,Medium,No,No,No,Fair,Low,Stayed,>50,>40
2,Female,Healthcare,Good,High,Low,0,No,Bachelors Degree,Married,3,Mid,Medium,No,No,No,Poor,Low,Stayed,23-30,8-18
3,Female,Education,Good,High,High,1,No,High School,Single,2,Mid,Small,Yes,No,No,Good,Medium,Stayed,30-40,23-30
4,Male,Education,Fair,Very High,Average,0,Yes,High School,Divorced,0,Senior,Medium,No,No,No,Fair,Medium,Stayed,>50,8-18


In [39]:
df.describe().round(2)

,Years at Company,Monthly Income,Distance from Home
count,74498.00,74498.00,74498.00
mean,15.72,7299.38,49.99
std,11.22,2152.51,28.51
min,1.00,1226.00,1.00
25%,7.00,5652.00,25.00
50%,13.00,7348.00,50.00
75%,23.00,8876.00,75.00
max,51.00,16149.00,99.00


In [41]:
df.shape

(74498, 23)

### Save as pickle file

In [44]:
folder = "data"
file_name = "processed_employee_data"

save_file_as_pickle(df, folder, file_name)

File saved as: ../data/processed_employee_data_20250325.pkl
